## Import required libraries

In [1]:
import time
from textwrap import dedent

from crewai import Crew, Agent, Task
from langchain_community.llms import Ollama

## Initialize LLM
We've chosen to use the llamma3 model for all use by all agents.

*More about the model here...*

In [2]:
llm = Ollama(model="llama3")

## Define Agents
We define an Agent using the following parameters:
- role: 
- goal: 
- backstory: 
- allow_delegation:
- verbose:
- llm:

In [3]:
AGENTS_DATA = {
    "senior_python_developer": {
        "role": "Senior Python Developer",
        "goal": dedent(
            """
            To review Python functions and ensure that variable names are
            following best practices. Variable names are concise, descriptive,
            and easy to understand.
            """
        ),
        "backstory": dedent(
            """
            You're a Python developer who's been programming for decades. You're
            renowned for writing excellent code, and particularly for writing
            high-quality variable names.
            """
        ),
    },
    "senior_code_documentation_expert": {
        "role": "Senior Code Documentation Expert",
        "goal": "Write docstrings for Python functions.",
        "backstory": dedent(
            """
            You're a renowned code documentation expert, specifically in Python.
            You know how to write clean, concise docstrings that include all the
            information relevant to developers trying to understand a function.
            """
        ),
    },
    "senior_python_qa_tester": {
        "role": "Senior Python Quality Assurance Tester",
        "goal": dedent(
            """
            To review Python code and ensure that it follows best practices.
            Double-check that variable names are accurate and concise and that
            the docstrings accurately reflect what the function actually does.
            """
        ),
        "backstory": dedent(
            """
            You're a renowned quality assurance expert. You've been reviewing code
            for decades and ensure that code is flawless, with excellent variable
            names and docstrings.
            """
        ),
    },
    "senior_code_writer": {
        "role": "Senior Code Writer",
        "goal": dedent(
            """
            Take code and strip out everything except the code
            and docstrings.
            """
        ),
        "backstory": dedent(
            """
            You do an amazing job of getting code and stripping it
            of everything that's not code or documentation.
            """
        ),
    },
}

# Function to create an Agent object
def create_agent(
    agent_type: str,
    llm: Ollama = llm,
    allow_delegation: bool = False,
    verbose: bool = False,
):
    agent_data = AGENTS_DATA.get(agent_type)
    if not agent_data:
        raise ValueError(f"No agent data found for type '{agent_type}'")
    return Agent(
        role=agent_data["role"],
        goal=agent_data["goal"],
        backstory=agent_data["backstory"],
        allow_delegation=allow_delegation,
        verbose=verbose,
        llm=llm,
    )

## Define Tasks
We define a Task using the following parameters:
- description:
- expected_output:
- agent:
- context:
- llm: 

In [4]:
TASKS_DATA = {
    "improve_variable_names": {
        "description": dedent(
            """
            Review the variable names in the Python code. Update
            the variable names so that they're accurate, concise,
            easy to read, and easy to understand. Update the variable 
            names only; don't change anything else about the functions, 
            and do not add comments anywhere in the code.

            Python Function
            ---------------
            {code}
            """
        ),
        "expected_output": "A Python function with variable names that are accurate, concise, and easy to understand.",
    },
    "add_docstrings": {
        "description": dedent(
            """
            Provide docstrings to the Python code provided below.
            Ensure the documentation is concise, accurate, and easy
            to read.
            """
        ),
        "expected_output": "A Python function with a well-written, concise, and easy to understand docstring.",
    },
    "review_code": {
        "description": dedent(
            """
            Review code and documentation and ensure it meets the
            highest standards of quality. The variable names are
            clear, concise, and accurate; the docstrings are
            accurate and descriptive, so that any developer reading
            them, even a junior developer, would understand exactly
            what they do and how to use them.
            """
        ),
        "expected_output": "A Python function with excellent variable names and a well-written docstring.",
    },
    "write_code": {
        "description": dedent(
            """
            Review the provided code and remove anything that
            is not code or docstrings.
            """
        ),
        "expected_output": "Code that contains ONLY functions and docstrings. All other information is removed.",
    },
}

# Function to create a Task object
def create_task(task_type, agent, code=None, context=None):
    data = TASKS_DATA.get(task_type)
    if not data:
        raise ValueError(f"No task data found for type '{task_type}'")
    description = data["description"]
    if code:
        description = description.format(code=code)
    return Task(
        description=description,
        agent=agent,
        expected_output=data["expected_output"],
        context=context or [],
    )

## Select sample code used as test input
We will use the following code (a merge sort algorith with poor variable names) as a sample input to test the Crew.

In [5]:
code = """
def m(a, b):
    c = []
    i, j = 0, 0
    while i < len(a) and j < len(b):
        if a[i] < b[j]:
            c.append(a[i])
            i += 1
        else:
            c.append(b[j])
            j += 1
    c.extend(a[i:])
    c.extend(b[j:])
    return c
"""

## Instantiate the Crew (Agents and Tasks)

In [6]:
def create_crew(code: str, verbose=0) -> Crew:
    """
    Create a Crew instance with agents and tasks for refactoring code.
    """

    # Create Agents
    variable_name_agent = create_agent("senior_python_developer")
    docstring_agent = create_agent("senior_code_documentation_expert")
    qa_agent = create_agent("senior_python_qa_tester")
    code_writer_agent = create_agent("senior_code_writer")

    # Create Tasks
    improve_variable_names_task = create_task(
        "improve_variable_names",
        agent=variable_name_agent,
        code=code,
    )
    add_docstrings_task = create_task(
        "add_docstrings",
        agent=docstring_agent,
        context=[improve_variable_names_task],
    )
    review_code_task = create_task(
        "review_code",
        agent=qa_agent,
        context=[add_docstrings_task],
    )
    write_code_task = create_task(
        "write_code",
        agent=code_writer_agent,
        context=[review_code_task],
    )

    return Crew(
        agents=[
            variable_name_agent,
            docstring_agent,
            qa_agent,
            code_writer_agent,
        ],
        tasks=[
            improve_variable_names_task,
            add_docstrings_task,
            review_code_task,
            write_code_task,
        ],
        verbose=verbose,
    )

crew = create_crew(code, verbose=1)

## Kickoff the crew to  begin processing tasks

In [7]:
start_time = time.perf_counter()
# Start Crew processing
result     = crew.kickoff()
end_time   = time.perf_counter()

print("###########################")
print("## Total Processing Time ##")
print("###########################")
print(f">>> {end_time - start_time:.2f} seconds <<<\n")

print("######################")
print("## Updated Function ##")
print("######################")
print(result)

 [DEBUG]: == Working Agent: Senior Python Developer
 [DEBUG]: == [Senior Python Developer] Task output: def merge_sorted_lists(left_list, right_list):
    merged_list = []
    left_index, right_index = 0, 0
    while left_index < len(left_list) and right_index < len(right_list):
        if left_list[left_index] < right_list[right_index]:
            merged_list.append(left_list[left_index])
            left_index += 1
        else:
            merged_list.append(right_list[right_index])
            right_index += 1
    merged_list.extend(left_list[left_index:])
    merged_list.extend(right_list[right_index:])
    return merged_list


 [DEBUG]: == Working Agent: Senior Code Documentation Expert
 [DEBUG]: == [Senior Code Documentation Expert] Task output: """
merge_sorted_lists(left_list, right_list)
-------------------------------------------------

Merges two sorted lists into one sorted list.

Parameters:
    left_list (list): The first sorted list to merge.
    right_list (list): The

## Compare with an ideal output
```python
def merge(sorted_list1, sorted_list2):
    """
    Merge two sorted lists into a single sorted list.

    Parameters:
        sorted_list1 (list): The first sorted list.
        sorted_list2 (list): The second sorted list.

    Returns:
        list: A merged and sorted list containing all elements from both input lists.
    """
    merged_list = []
    index1, index2 = 0, 0
    while index1 < len(sorted_list1) and index2 < len(sorted_list2):
        if sorted_list1[index1] < sorted_list2[index2]:
            merged_list.append(sorted_list1[index1])
            index1 += 1
        else:
            merged_list.append(sorted_list2[index2])
            index2 += 1
    merged_list.extend(sorted_list1[index1:])
    merged_list.extend(sorted_list2[index2:])
    return merged_list
```